# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902025


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:40,  1.43s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:36,  1.42s/it]

Rendering models:  17%|█▋        | 5/30 [00:05<00:25,  1.04s/it]

Rendering models:  20%|██        | 6/30 [00:05<00:18,  1.28it/s]

...

Rendering models:  87%|████████▋ | 26/30 [00:10<00:00,  6.19it/s]

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  6.44it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  5.73it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  5.81it/s]

Jes8x                                 0.000930
kayleebug2017                         0.000956
not-logged-in-2e775991bd3ca814ca1d    0.009773
equidad1                              0.065900
CTidwell3                             0.000874
Mouring                               0.000758
klmasters                             0.000510
Linda_J._Berkel                       0.000495
Vorealus                              0.000535
not-logged-in-03b60317d6ee47942c58    0.001104
lsautter                              0.000487
not-logged-in-a8038c9112f1e417d868    0.021987
not-logged-in-9a813f8c400d5fedde5d    0.002544
not-logged-in-9e6c824286712141ea56    0.002270
not-logged-in-af5cbed4d995f3b9cb68    0.002587
pangeli5                              0.000802
acapirala                             0.005761
not-logged-in-300dca7f45d1a5d1dd58    0.023310
not-logged-in-fcfcbc88feb1aa0ed0a7    0.001126
ShirleyL                              0.000719
not-logged-in-fb214ee43db59c93f8b1    0.004663
Denny2507    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())